<a href="https://colab.research.google.com/github/SHIVASAI16256/GEN-AI-B-38/blob/main/2303A52488_GEN_AI_ASS_11_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Function to build the CNN model with the provided architecture
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolution Layer 1 + Max Pooling Layer 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolution Layer 2 + Max Pooling Layer 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolution Layer 3 + Max Pooling Layer 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output for the dense layer
    model.add(layers.Flatten())

    # Dense Layer
    model.add(layers.Dense(256, activation='relu'))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Function to load images from folders
def load_cat_dog_data(train_dir, validation_dir, img_height=150, img_width=150, batch_size=32):
    print("Loading cat and dog dataset...")

    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Only rescaling for validation
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Load training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Load validation data
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_generator, validation_generator

# Function to train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    # Compile model with Adadelta optimizer as specified
    model.compile(
        optimizer='adadelta',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator
    )

    return history

# Function to evaluate the model
def evaluate_model(model, validation_generator):
    # Evaluate the model on the validation data
    validation_steps = validation_generator.samples // validation_generator.batch_size
    evaluation = model.evaluate(validation_generator, steps=validation_steps)

    test_loss, test_acc = evaluation[0], evaluation[1]
    print(f"\nValidation accuracy: {test_acc:.4f}")
    print(f"Validation loss: {test_loss:.4f}")

    # Get predictions
    validation_steps = validation_generator.samples // validation_generator.batch_size + 1
    predictions = model.predict(validation_generator, steps=validation_steps)
    predicted_classes = np.argmax(predictions[:validation_generator.samples], axis=1)
    true_classes = validation_generator.classes[:validation_generator.samples]

    # Compute confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)

    return test_acc, test_loss, cm, true_classes, predicted_classes

# Function to plot training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Function to create and evaluate alternative architectures
def experiment_with_architectures(train_generator, validation_generator, input_shape, num_classes):
    architectures = {
        "Original": {
            "conv_filters": [64, 128, 256],
            "dense_units": [256]
        },
        "Deeper": {
            "conv_filters": [32, 64, 128, 256],
            "dense_units": [512, 256]
        },
        "Wider": {
            "conv_filters": [128, 256, 512],
            "dense_units": [512]
        },
        "Balanced": {
            "conv_filters": [64, 128, 256, 256],
            "dense_units": [512, 256]
        }
    }

    results = {}

    for name, arch in architectures.items():
        print(f"\n\n{'='*50}")
        print(f"Training architecture: {name}")
        print(f"{'='*50}")

        model = models.Sequential()

        # Add convolutional layers
        for i, filters in enumerate(arch["conv_filters"]):
            if i == 0:
                model.add(layers.Conv2D(filters, (3, 3), activation='relu', input_shape=input_shape))
            else:
                model.add(layers.Conv2D(filters, (3, 3), activation='relu'))
            model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Flatten())

        # Add dense layers
        for units in arch["dense_units"]:
            model.add(layers.Dense(units, activation='relu'))

        # Output layer
        model.add(layers.Dense(num_classes, activation='softmax'))

        model.summary()

        # Train the model
        history = train_model(model, train_generator, validation_generator)

        # Evaluate the model
        validation_steps = validation_generator.samples // validation_generator.batch_size
        eval_result = model.evaluate(validation_generator, steps=validation_steps)
        test_loss, test_acc = eval_result[0], eval_result[1]
        train_acc = history.history['accuracy'][-1]

        # Store results
        results[name] = {
            "train_accuracy": train_acc,
            "validation_accuracy": test_acc,
            "validation_loss": test_loss
        }

    # Compare results
    print("\n\n")
    print("="*80)
    print("Architecture Comparison")
    print("="*80)
    print(f"{'Architecture':<15} {'Train Accuracy':<20} {'Validation Accuracy':<20} {'Validation Loss':<15}")
    print("-"*80)

    for arch, metrics in results.items():
        print(f"{arch:<15} {metrics['train_accuracy']:.4f}{' '*12} {metrics['validation_accuracy']:.4f}{' '*12} {metrics['validation_loss']:.4f}")

    # Identify the best architecture
    best_arch = max(results.items(), key=lambda x: x[1]['validation_accuracy'])
    print("\nBest architecture based on validation accuracy:")
    print(f"{best_arch[0]} with validation accuracy of {best_arch[1]['validation_accuracy']:.4f}")

# Main execution
def main():
    # Set your dataset directories here
    train_dir = '/content/drive/MyDrive/GEN AI/train'  # Directory with 'cat' and 'dog' subdirectories of training images
    validation_dir = '/content/drive/MyDrive/GEN AI/validation-20250404T145130Z-001'  # Directory with 'cat' and 'dog' subdirectories of validation images

    img_height = 150
    img_width = 150
    batch_size = 32
    epochs = 10

    # Check if directories exist
    if not os.path.exists(train_dir):
        print(f"Error: Training directory '{train_dir}' not found!")
        return
    if not os.path.exists(validation_dir):
        print(f"Error: Validation directory '{validation_dir}' not found!")
        return

    # Load data
    train_generator, validation_generator = load_cat_dog_data(
        train_dir, validation_dir, img_height, img_width, batch_size
    )

    # Set input shape and number of classes
    input_shape = (img_height, img_width, 3)
    num_classes = len(train_generator.class_indices)  # Should be 2 for cats and dogs

    print(f"Number of classes detected: {num_classes}")
    print(f"Class mapping: {train_generator.class_indices}")

    # Build the CNN model with the provided architecture
    print("\nBuilding the CNN model...")
    model = build_cnn_model(input_shape, num_classes)
    model.summary()

    # Train the model
    print("\nTraining the model...")
    history = train_model(model, train_generator, validation_generator, epochs=epochs)

    # Evaluate the model
    print("\nEvaluating the model...")
    test_acc, test_loss, cm, true_classes, pred_classes = evaluate_model(model, validation_generator)

    # Plot training history
    plot_training_history(history)

    # Plot confusion matrix
    class_names = list(train_generator.class_indices.keys())
    plot_confusion_matrix(cm, class_names)

    # Save the model
    model.save('cat_dog_cnn_model.h5')
    print("Model saved as 'cat_dog_cnn_model.h5'")

    # Experiment with different architectures
    print("\nExperimenting with different architectures...")
    experiment_with_architectures(train_generator, validation_generator, input_shape, num_classes)

if __name__ == "_main_":
    main()

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Function to build the CNN model with the provided architecture
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolution Layer 1 + Max Pooling Layer 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolution Layer 2 + Max Pooling Layer 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolution Layer 3 + Max Pooling Layer 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output for the dense layer
    model.add(layers.Flatten())

    # Dense Layer
    model.add(layers.Dense(256, activation='relu'))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Function to load images from folders
def load_cat_dog_data(train_dir, validation_dir, img_height=150, img_width=150, batch_size=32):
    print("Loading cat and dog dataset...")

    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Only rescaling for validation
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Load training data
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Load validation data
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_generator, validation_generator

# Function to train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    # Compile model with Adadelta optimizer as specified
    model.compile(
        optimizer='adadelta',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator
    )

    return history

# Function to evaluate the model
def evaluate_model(model, validation_generator):
    # Evaluate the model on the validation data
    validation_steps = validation_generator.samples // validation_generator.batch_size
    evaluation = model.evaluate(validation_generator, steps=validation_steps)

    test_loss, test_acc = evaluation[0], evaluation[1]
    print(f"\nValidation accuracy: {test_acc:.4f}")
    print(f"Validation loss: {test_loss:.4f}")

    # Get predictions
    validation_steps = validation_generator.samples // validation_generator.batch_size + 1
    predictions = model.predict(validation_generator, steps=validation_steps)
    predicted_classes = np.argmax(predictions[:validation_generator.samples], axis=1)
    true_classes = validation_generator.classes[:validation_generator.samples]

    # Compute confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)

    return test_acc, test_loss, cm, true_classes, predicted_classes

# Function to plot training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Function to create and evaluate alternative architectures
def experiment_with_architectures(train_generator, validation_generator, input_shape, num_classes):
    architectures = {
        "Original": {
            "conv_filters": [64, 128, 256],
            "dense_units": [256]
        },
        "Deeper": {
            "conv_filters": [32, 64, 128, 256],
            "dense_units": [512, 256]
        },
        "Wider": {
            "conv_filters": [128, 256, 512],
            "dense_units": [512]
        },
        "Balanced": {
            "conv_filters": [64, 128, 256, 256],
            "dense_units": [512, 256]
        }
    }

    results = {}

    for name, arch in architectures.items():
        print(f"\n\n{'='*50}")
        print(f"Training architecture: {name}")
        print(f"{'='*50}")

        model = models.Sequential()

        # Add convolutional layers
        for i, filters in enumerate(arch["conv_filters"]):
            if i == 0:
                model.add(layers.Conv2D(filters, (3, 3), activation='relu', input_shape=input_shape))
            else:
                model.add(layers.Conv2D(filters, (3, 3), activation='relu'))
            model.add(layers.MaxPooling2D((2, 2)))

        model.add(layers.Flatten())

        # Add dense layers
        for units in arch["dense_units"]:
            model.add(layers.Dense(units, activation='relu'))

        # Output layer
        model.add(layers.Dense(num_classes, activation='softmax'))

        model.summary()

        # Train the model
        history = train_model(model, train_generator, validation_generator)

        # Evaluate the model
        validation_steps = validation_generator.samples // validation_generator.batch_size
        eval_result = model.evaluate(validation_generator, steps=validation_steps)
        test_loss, test_acc = eval_result[0], eval_result[1]
        train_acc = history.history['accuracy'][-1]

        # Store results
        results[name] = {
            "train_accuracy": train_acc,
            "validation_accuracy": test_acc,
            "validation_loss": test_loss
        }

    # Compare results
    print("\n\n")
    print("="*80)
    print("Architecture Comparison")
    print("="*80)
    print(f"{'Architecture':<15} {'Train Accuracy':<20} {'Validation Accuracy':<20} {'Validation Loss':<15}")
    print("-"*80)

    for arch, metrics in results.items():
        print(f"{arch:<15} {metrics['train_accuracy']:.4f}{' '*12} {metrics['validation_accuracy']:.4f}{' '*12} {metrics['validation_loss']:.4f}")

    # Identify the best architecture
    best_arch = max(results.items(), key=lambda x: x[1]['validation_accuracy'])
    print("\nBest architecture based on validation accuracy:")
    print(f"{best_arch[0]} with validation accuracy of {best_arch[1]['validation_accuracy']:.4f}")

# Main execution
def main():
    # Set your dataset directories here
    train_dir = '/content/drive/MyDrive/train-20250404T145129Z-001/train'  # Directory with 'cat' and 'dog' subdirectories of training images
    validation_dir = '/content/drive/MyDrive/validation-20250404T145130Z-001/validation'  # Directory with 'cat' and 'dog' subdirectories of validation images

    img_height = 150
    img_width = 150
    batch_size = 32
    epochs = 10

    # Check if directories exist
    if not os.path.exists(train_dir):
        print(f"Error: Training directory '{train_dir}' not found!")
        return
    if not os.path.exists(validation_dir):
        print(f"Error: Validation directory '{validation_dir}' not found!")
        return

    # Load data
    train_generator, validation_generator = load_cat_dog_data(
        train_dir, validation_dir, img_height, img_width, batch_size
    )

    # Set input shape and number of classes
    input_shape = (img_height, img_width, 3)
    num_classes = len(train_generator.class_indices)  # Should be 2 for cats and dogs

    print(f"Number of classes detected: {num_classes}")
    print(f"Class mapping: {train_generator.class_indices}")

    # Build the CNN model with the provided architecture
    print("\nBuilding the CNN model...")
    model = build_cnn_model(input_shape, num_classes)
    model.summary()

    # Train the model
    print("\nTraining the model...")
    history = train_model(model, train_generator, validation_generator, epochs=epochs)

    # Evaluate the model
    print("\nEvaluating the model...")
    test_acc, test_loss, cm, true_classes, pred_classes = evaluate_model(model, validation_generator)

    # Plot training history
    plot_training_history(history)

    # Plot confusion matrix
    class_names = list(train_generator.class_indices.keys())
    plot_confusion_matrix(cm, class_names)

    # Save the model
    model.save('cat_dog_cnn_model.h5')
    print("Model saved as 'cat_dog_cnn_model.h5'")

    # Experiment with different architectures
    print("\nExperimenting with different architectures...")
    experiment_with_architectures(train_generator, validation_generator, input_shape, num_classes)

if __name__ == "_main_":
    main()